# Zielstellung

- Mithilfe des Codes aus dieser Datei sollen verschiedene generelle Raumanalysen möglich gemacht werden
- dabei werden für bestimmte Punkte (Geokoordinaten) bestimmte Werte ermittelt, die anschließend auf einer heatmap farblich dargestellt werden


# Umwandeln der Koordinaten in ein besser nutzbares Format

- Die Geokoordinateni n WSG84 werden in das für Deutschland geeignete epsg:31468-Format umgewandelt
- von nun an können sie wie Angaben in Metern betrachtet werden, insbesondere die Abstandsberechnung wird dadurch deutlich vereinfacht

In [8]:
import json
from tqdm import tqdm

def extract_coordinates():
    from pyproj import Proj
    data = {}
    with open("gtfsmdvlvb/stops.txt", "r") as f:
        txt = [elem.split(",") for elem in f.read().split("\n")[1:-1]]
    for elem in tqdm(txt):
        #data[elem[0]] = [float(elem[-2]), float(elem[-1])]
        data[elem[0]] = Proj('epsg:31468')(float(elem[-1]), float(elem[-2]))
    with open("lvb_auswertung/coordinates.json", "w+") as f:
        json.dump(data, f, indent=4)
    return data

def load_coordinates():
    with open("lvb_auswertung/coordinates.json", "r") as f:
        return json.load(f)

def min_max():
    data = load_coordinates()
    x = [data[elem][0] for elem in data]
    y = [data[elem][1] for elem in data]
    return [(min(x), max(x)), (min(y), max(y))]

# gibt Zwischenschritte zwischen Extremkoordinaten abhängig von einer Distanz für ein grid zurück
def steps(dist):
    m_m = min_max()
    cur_x = m_m[0][0]
    cur_y = m_m[1][0]
    x_steps, y_steps = [cur_x], [cur_y]
    while cur_x < m_m[0][1]:
        cur_x += dist
        x_steps.append(cur_x)
    while cur_y < m_m[1][1]:
        cur_y += dist
        y_steps.append(cur_y)
    return [x_steps, y_steps]


#extract_coordinates()
#print(min_max())
c = steps(500)
print(len(c[0])*len(c[1]))

48705


# Finden von Haltestellendichten an einem Punkt

In [21]:
from copy import deepcopy

# for every point, we execute this function, that gives back a value based on the distances to all other points
# TODO: Gewichtung eines Punktes (zum Beispiel relevant für Anzahl der Verbindungen an einer Haltestelle -> könnte in die Haltestellendichte einfließen)
def all_distances_point(coordinate, cur_haltestellen):
    cur_haltestellen.remove(coordinate)
    distances = []
    for other_coordinate in cur_haltestellen:
        #if coordinate == other_coordinate:
        #    continue
        coordinate, other_coordinate = tuple(coordinate), tuple(other_coordinate)
        distances.append(
            #GD(coordinate, other_coordinate).m
            ((coordinate[0]-other_coordinate[0])**2+(coordinate[0]-other_coordinate[0])**2)**(1/2)
        )
    if 0 in distances:
        distances = distances.remove(0)
    if distances is None:
        return 0
    value = sum([1/x for x in distances])
    return value

# Hier müssen wir nochmal überlegen, wie wir die Punkte ausdrücken wollen (idealerweise nicht durch die Koordinaten, sondern durch stop-indizes beziehungsweise Rasterkoordinaten (ganzzahlig als Tupel bzw string im format '{x}|{y}'))
def all_distances(points, haltestellen):
    distances = []
    for point in tqdm(points):
        distances.append([point, all_distances_point(points[point], deepcopy(haltestellen))])
    return distances

def haltestellendichte():
    with open("lvb_auswertung/coordinates.json", "r") as f:
        data = json.load(f)
        coordinates = [data[elem] for elem in data]
    
    with open("lvb_auswertung/dichte.json", "w+") as f:
        json.dump(all_distances(data, coordinates), f, indent=4)


haltestellendichte()

100%|██████████| 4377/4377 [00:46<00:00, 93.83it/s] 
